In [1]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, roc_curve
from sklearn.utils import class_weight
import numpy as np

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.25  # Increased validation split
)

train_generator = train_datagen.flow_from_directory(
    '/Users/gagan/Desktop/University /chest_xray/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    '/Users/gagan/Desktop/University /chest_xray/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

# Load ResNet50
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers initially
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Calculate dynamic class weights
class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# Callbacks
lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1)
early_stop = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[lr_reduce, early_stop],
    class_weight=class_weights
)

# Fine-tuning: Unfreeze the last 50 layers of the base model
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=Adam(learning_rate=1e-6), loss='binary_crossentropy', metrics=['accuracy'])

# Fine-tune the model for additional epochs
history_finetune = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=[lr_reduce, early_stop],
    class_weight=class_weights
)

# Save the fine-tuned model
model.save('/Users/gagan/Desktop/University/pneumonia_detection_resnet50_optimized_3.h5')

# Evaluate the model on the validation set
validation_generator.reset()
preds = model.predict(validation_generator)
preds = preds.flatten()

# ROC Curve analysis to find the optimal threshold
fpr, tpr, thresholds = roc_curve(validation_generator.classes, preds)
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]
preds_optimal = (preds > optimal_threshold).astype(int)

# Classification report and confusion matrix
print(classification_report(validation_generator.classes, preds_optimal, target_names=['Normal', 'Pneumonia']))
print(confusion_matrix(validation_generator.classes, preds_optimal))

Found 3913 images belonging to 2 classes.
Found 1303 images belonging to 2 classes.
Epoch 1/50


/Users/gagan/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


123/123 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.5052 - loss: 0.7552 - val_accuracy: 0.7460 - val_loss: 0.5968 - learning_rate: 1.0000e-04
Epoch 2/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.5865 - loss: 0.6805 - val_accuracy: 0.7421 - val_loss: 0.6175 - learning_rate: 1.0000e-04
Epoch 3/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 152s 1s/step - accuracy: 0.6734 - loss: 0.6170 - val_accuracy: 0.6025 - val_loss: 0.6830 - learning_rate: 1.0000e-04
Epoch 4/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 923ms/step - accuracy: 0.6979 - loss: 0.5938
Epoch 4: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
123/123 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.6978 - loss: 0.5939 - val_accuracy: 0.7229 - val_loss: 0.6308 - learning_rate: 1.0000e-04
Epoch 5/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - accuracy: 0.7229 - loss: 0.5793 - val_accuracy: 0.7176 - val_loss: 0.5992 - learning_rate: 1.0000e-05
Epoch 6/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 162s 1s/step - accuracy: 0.7146 -

41/41 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step
              precision    recall  f1-score   support

      Normal       0.26      0.91      0.40       335
   Pneumonia       0.77      0.11      0.19       968

    accuracy                           0.31      1303
   macro avg       0.51      0.51      0.29      1303
weighted avg       0.64      0.31      0.24      1303

[[304  31]
 [866 102]]


In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# Path to the dataset directories
test_dir ='/Users/gagan/Desktop/University /chest_xray/test'
# Load your model
model = load_model('/Users/gagan/Desktop/University/Final submit/pneumonia_detection_resnet50_optimized_3.h5')

# Set up data augmentation and preprocessing for the test data
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  # Ensure this matches your model's input size
    batch_size=16,
    class_mode='binary',  # Binary classification since you only have two classes
    shuffle=False  # Do not shuffle the test set
)

# Compile the model with binary crossentropy and appropriate metrics
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Binary crossentropy for binary classification
              metrics=['accuracy'])

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator)
print(f'Test Loss: {loss}')
print(f'Test Accuracy: {accuracy}')

# Get predictions
predictions = model.predict(test_generator)
predicted_classes = (predictions > 0.5).astype("int32").flatten()  # Convert probabilities to binary class predictions

# True labels
true_classes = test_generator.classes
class_labels = list(test_generator.class_indices.keys())

# Print classification report
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

# Confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print('Confusion Matrix:')
print(conf_matrix)

# Optional: Apply ReduceLROnPlateau to improve training (if you retrain the model)
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)
# Example for retraining (adjust according to your training setup)
# model.fit(train_generator, validation_data=validation_generator, epochs=10, class_weight=class_weights_dict, callbacks=[reduce_lr])

Found 624 images belonging to 2 classes.


/Users/gagan/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


39/39 ━━━━━━━━━━━━━━━━━━━━ 18s 433ms/step - accuracy: 0.8307 - loss: 0.4874
Test Loss: 0.399677574634552
Test Accuracy: 0.8445512652397156
39/39 ━━━━━━━━━━━━━━━━━━━━ 17s 433ms/step
              precision    recall  f1-score   support

      NORMAL       0.79      0.79      0.79       234
   PNEUMONIA       0.87      0.88      0.88       390

    accuracy                           0.84       624
   macro avg       0.83      0.83      0.83       624
weighted avg       0.84      0.84      0.84       624

Confusion Matrix:
[[185  49]
 [ 48 342]]
